# 5 - Vertex AI + BQML - Online Predictions with BQML Models

Models built with BigQuery ML (BQML), like the one in (03a), can also be exported for use and deployment outside of BigQuery.  A Vertex AI Endpoint can be used for online predictions with an exported model.  This demonstration shows the process of exporting and deploying a BQML model with Vertex AI.



### Prerequisites:
-  03a - BigQuery Machine Learning (BQML) - Machine Learning with SQL

### Overview:
-  Export the BigQuery ML model built in (03a) to a GCS bucket URI
   -  EXPORT MODEL …
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Upload Model
      -  Model - aiplatform.Model.upoad
   -  Create Endpoint
      -  Endpoint - aiplatform.Endpoint.create
   -  Deploy to Endpoint
      -  Endpoint.deploy(model=Model)
   -  Online Predictions
      -  Endpoint.predict

### Resources:
-  [Export formats for BigQuery ML models](https://cloud.google.com/bigquery-ml/docs/exporting-models)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)


---
## Setup

inputs:

In [1]:
REGION = 'us-central1'
PROJECT_ID='qwiklabs-gcp-04-24efb7132888'
DATANAME = 'fraud'
NOTEBOOK = '5'

# Resources
DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [2]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [4]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = "conchitademomlopsendtoend"
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
params = {"URI": URI}
DIR = f"temp/{NOTEBOOK}"

environment:

In [5]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Export the BigQuery Model

Export the BigQuery Model:
- https://cloud.google.com/bigquery-ml/docs/exporting-models

In [6]:
export = bigquery.query(query = f"EXPORT MODEL {DATANAME}.{DATANAME}_lr OPTIONS(URI = '{URI}')")

In [7]:
export.result()

---
## Serving

### Upload The Model
https://googleapis.dev/python/aiplatform/latest/aiplatform.html?highlight=aiplatform%20model%20upload#google.cloud.aiplatform.Model.upload

In [8]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = URI,
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/948421954571/locations/us-central1/models/5499198610228641792/operations/2509310453432188928
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/948421954571/locations/us-central1/models/5499198610228641792
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/948421954571/locations/us-central1/models/5499198610228641792')


In [9]:
model.display_name

'5_fraud_20220320115846'

### Create An Endpoint

In [10]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/948421954571/locations/us-central1/endpoints/230149773926072320/operations/3824361544624373760
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/948421954571/locations/us-central1/endpoints/230149773926072320
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/948421954571/locations/us-central1/endpoints/230149773926072320')


In [11]:
endpoint.display_name

'5_fraud_20220320115846'

### Deploy Model To Endpoint

In [12]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/948421954571/locations/us-central1/models/5499198610228641792 to Endpoint : projects/948421954571/locations/us-central1/endpoints/230149773926072320
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/948421954571/locations/us-central1/endpoints/230149773926072320/operations/7769514818200928256
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/948421954571/locations/us-central1/endpoints/230149773926072320


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [13]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [14]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,108552,-0.800747,1.185243,-1.307864,-1.021146,-0.501487,-0.280646,-1.055654,0.994632,2.277438,...,0.244974,0.081168,-1.856601,-1.159921,-0.300115,0.172426,0.0,0,683f3632-4d23-46b1-bc28-bdea77438e0a,TEST
1,152933,1.899377,0.520212,-0.397464,3.492509,0.660067,0.902600,-0.126600,0.209196,-1.032307,...,0.509313,0.172632,-0.523424,-0.628977,-0.015943,-0.037275,0.0,0,9b621701-7e1d-44ee-b83f-0872c3feac62,TEST
2,169514,1.983349,-0.319376,-0.391078,0.287527,-0.449550,-0.166502,-0.592739,0.006890,0.962290,...,0.014298,-0.269116,0.023498,-0.142238,0.042028,-0.050875,0.0,0,7c673636-c18d-4c43-895a-eeadd92ef1ef,TEST
3,56774,1.282808,0.153264,0.111244,0.682014,-0.362113,-1.027642,0.192959,-0.227975,0.264107,...,-0.098736,0.442745,0.643293,0.556626,-0.056407,-0.000733,0.0,0,e5045737-d174-4678-a09a-4c66882fc1fa,TEST


In [15]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET,'splits'])]].to_dict(orient='records')[0]
#newob

In [16]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [17]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[{'Class_values': ['1', '0'], 'predicted_Class': ['0'], 'Class_probs': [0.0166778101445207, 0.9833221898554793]}], deployed_model_id='1323169885051682816', explanations=None)

In [18]:
prediction.predictions[0]#['classes'][np.argmax(prediction.predictions[0]['scores'])]

{'Class_values': ['1', '0'],
 'predicted_Class': ['0'],
 'Class_probs': [0.0166778101445207, 0.9833221898554793]}

In [19]:
prediction.predictions[0][f'{VAR_TARGET}_values'][np.argmax(prediction.predictions[0][f'{VAR_TARGET}_probs'])]

'0'

### Get Predictions: REST

In [20]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [21]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

E0320 12:05:33.050984586   30433 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


{
  "predictions": [
    {
      "Class_probs": [
        0.0166778101445207,
        0.98332218985547926
      ],
      "predicted_Class": [
        "0"
      ],
      "Class_values": [
        "1",
        "0"
      ]
    }
  ],
  "deployedModelId": "1323169885051682816",
  "model": "projects/948421954571/locations/us-central1/models/5499198610228641792",
  "modelDisplayName": "5_fraud_20220320115846"
}


### Get Predictions: gcloud (CLI)

In [22]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

E0320 12:05:34.106787416   30433 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[{'Class_probs': [0.0166778101445207, 0.9833221898554793], 'Class_values': ['1', '0'], 'predicted_Class': ['0']}]
